In [18]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number
import geopandas 
import editdistance
import itertools
# Enable inline plotting
%matplotlib inline

In [19]:
filename = 'reduced_full_data.csv'
df = pd.read_csv(filename, dtype=object)

In [170]:
florida_df = df[df['contbr_st'] == 'OH'].copy()
florida_df['contb_receipt_amt'] = pd.to_numeric(florida_df['contb_receipt_amt'])
florida_df.groupby('cand_nm')['contb_receipt_amt'].sum().sort_values(ascending=False)

cand_nm
Kasich, John R.              4577712.57
Clinton, Hillary Rodham      2422294.09
Cruz, Rafael Edward 'Ted'    1395722.60
Sanders, Bernard             1376764.10
Rubio, Marco                 1005987.76
Carson, Benjamin S.           731006.55
Bush, Jeb                     260855.00
Paul, Rand                    117148.92
Trump, Donald J.              101432.14
Fiorina, Carly                 83575.09
Walker, Scott                  42158.00
Graham, Lindsey O.             23300.00
O'Malley, Martin Joseph        16870.00
Santorum, Richard J.           13964.60
Huckabee, Mike                 10651.00
Johnson, Gary                   8471.10
Christie, Christopher J.        6200.00
Lessig, Lawrence                4012.77
Jindal, Bobby                   3420.16
Perry, James R. (Rick)          3185.00
Webb, James Henry Jr.           2600.00
Stein, Jill                     1750.00
Pataki, George E.               1250.00
Name: contb_receipt_amt, dtype: float64

In [20]:
df.head()

,Unnamed: 0,contbr_st,cand_nm,contbr_nm_da,contbr_zip,contbr_employer,contb_receipt_amt,contb_receipt_dt
0,0,AE,"Rubio, Marco","RUCINSKI, ROBERT",09096,US ARMY,100,08-MAR-16
1,1,AE,"Rubio, Marco","DOHERTY, THERESA",09752,DODEA/DS,500,26-JUN-15
2,2,AE,"Rubio, Marco","MARQUEZ, PANFILO",09777,DEPARTMENT OF STATE,100,07-MAR-16
3,3,AE,"Rubio, Marco","RECKER, ANGELA",00138,GOVERNMENT,388.25,18-MAR-16
4,4,AE,"Rubio, Marco","RECKER, ANGELA",09012,GOVERNMENT,500,09-FEB-16


In [21]:
df.shape

(3759428, 8)

In [11]:
wo_na = df.dropna(subset=['contbr_employer'])
univs_df = wo_na[wo_na['contbr_employer'].str.contains('UNIV')].copy()

In [12]:
univs_df.head()

,Unnamed: 0,contbr_st,cand_nm,contbr_nm_da,contbr_zip,contbr_employer,contb_receipt_amt,contb_receipt_dt
267,277,AL,"Rubio, Marco","BIEBIGHAUSER, VICTOR",36111,SOUTH UNIVERSITY OF ALABAMA INC.,2700,21-JUN-15
437,453,AL,"Rubio, Marco","CARTER, JACK DR. JR.",35242,UNIVERSITY OF ALABAMA @ BIRMINGHAM,2000,11-DEC-15
929,953,AL,"Rubio, Marco","KOSHUT, TOM",35763,UNIVERSITY OF ALABAMA IN HUNTSVILLE,250,29-FEB-16
1386,1426,AR,"Rubio, Marco","SMITH, PAUL",72761,JOHN BROWN UNIVERSITY,50,11-FEB-16
1387,1427,AR,"Rubio, Marco","SMITH, PAUL",72761,JOHN BROWN UNIVERSITY,100,06-FEB-16


In [13]:
univs_df.shape[0]

110334

In [17]:
len(univs_df.groupby(['contbr_nm_da', 'contbr_zip']))

17652

In [25]:
import us
states = [x.abbr for x in us.states.STATES]
states[:2]

[u'AL', u'AK']

In [54]:
valid_states_df = df[df['contbr_st'].isin(states)]
reduced_vs_df = valid_states_df[['contbr_st', 'cand_nm', 'contb_receipt_amt']].copy()
reduced_vs_df.head()

,contbr_st,cand_nm,contb_receipt_amt
18,AK,"Rubio, Marco",500
19,AK,"Rubio, Marco",100
20,AK,"Rubio, Marco",25
21,AK,"Rubio, Marco",250
22,AK,"Rubio, Marco",15


In [55]:
affil = pd.read_csv('candidates.csv', delimiter='|', header=None)
affil.columns = ['affil', 'name']
affil

,affil,name
0,R,"Bush, Jeb"
1,R,"Carson, Benjamin S."
2,R,"Christie, Christopher J."
3,D,"Clinton, Hillary Rodham"
4,R,"Cruz, Rafael Edward 'Ted'"
5,R,"Fiorina, Carly"
6,R,"Gilmore, James S IIII"
7,R,"Graham, Lindsey O."
8,R,"Huckabee, Mike"
9,R,"Jindal, Bobby"


In [56]:
affil_dict = affil.set_index('name').to_dict()['affil']
reduced_vs_df['cand_affil'] = reduced_vs_df['cand_nm'].map(affil_dict)
reduced_vs_df.head()

,contbr_st,cand_nm,contb_receipt_amt,cand_affil
18,AK,"Rubio, Marco",500,R
19,AK,"Rubio, Marco",100,R
20,AK,"Rubio, Marco",25,R
21,AK,"Rubio, Marco",250,R
22,AK,"Rubio, Marco",15,R


In [79]:
affil_df = reduced_vs_df[['contbr_st', 'contb_receipt_amt', 'cand_affil']].copy()
affil_df['contb_receipt_amt'] = pd.to_numeric(affil_df['contb_receipt_amt'])
affil_df.head()

,contbr_st,contb_receipt_amt,cand_affil
18,AK,500.0,R
19,AK,100.0,R
20,AK,25.0,R
21,AK,250.0,R
22,AK,15.0,R


In [83]:
summed = affil_df.groupby(['contbr_st', 'cand_affil']).sum().reset_index()
summed.head(9)

,contbr_st,cand_affil,contb_receipt_amt
0,AK,D,536517.78
1,AK,I,2775.40
2,AK,R,401026.80
3,AL,D,959516.96
4,AL,I,4300.00
5,AL,R,2528889.82
6,AR,D,1462504.68
7,AR,I,1904.59
8,AR,R,1660523.11


In [88]:
idx = summed.groupby('contbr_st')['contb_receipt_amt'].transform(max) == summed['contb_receipt_amt']
state_affil_df = summed[idx].copy()
state_affil_df.head()

,contbr_st,cand_affil,contb_receipt_amt
0,AK,D,5.365178e+05
5,AL,R,2.528890e+06
8,AR,R,1.660523e+06
11,AZ,R,4.008656e+06
12,CA,D,6.012483e+07


In [156]:
state_republ = summed[summed['cand_affil'] == 'R'].sort_values(by=['contbr_st']).reset_index(drop=True)
state_republ.head()

,contbr_st,cand_affil,contb_receipt_amt
0,AK,R,401026.80
1,AL,R,2528889.82
2,AR,R,1660523.11
3,AZ,R,4008656.36
4,CA,R,26993521.09


In [157]:
state_dem = summed[summed['cand_affil'] == 'D'].sort_values(by=['contbr_st']).reset_index(drop=True)
state_dem.head()

,contbr_st,cand_affil,contb_receipt_amt
0,AK,D,5.365178e+05
1,AL,D,9.595170e+05
2,AR,D,1.462505e+06
3,AZ,D,3.090719e+06
4,CA,D,6.012483e+07


In [134]:
diff_df = state_dem.copy()
diff_df['dem_surplus'] = state_dem['contb_receipt_amt'].subtract(state_republ['contb_receipt_amt'])
diff_df = diff_df[['contbr_st', 'dem_surplus']]
diff_df.apply(pd.scale)

AttributeError: 'module' object has no attribute 'scale'

In [158]:
def normalize(df):
    result = df.copy()
    #max_value = df['dem_surplus'].max()
    #min_value = df['dem_surplus'].min()
    #result['dem_surplus'] = (df['dem_surplus'] - min_value) / (max_value - min_value)
    return result

norm_df = normalize(diff_df)
norm_df.head(9)

,contbr_st,dem_surplus
0,AK,1.354910e+05
1,AL,-1.569373e+06
2,AR,-1.980184e+05
3,AZ,-9.179369e+05
4,CA,3.313131e+07
5,CO,2.234793e+06
6,CT,1.658298e+06
7,DC,5.922831e+06
8,DE,9.031073e+04


In [159]:
mappings = dict((x, y.lower()) for x, y in us.states.mapping('abbr', 'name').iteritems())
norm_df['contbr_st'] = norm_df['contbr_st'].map(mappings)

In [160]:
norm_df.columns = ['region', 'value']
norm_df.head(10)

,region,value
0,alaska,1.354910e+05
1,alabama,-1.569373e+06
2,arkansas,-1.980184e+05
3,arizona,-9.179369e+05
4,california,3.313131e+07
5,colorado,2.234793e+06
6,connecticut,1.658298e+06
7,district of columbia,5.922831e+06
8,delaware,9.031073e+04
9,florida,-9.831659e+06


In [161]:
norm_df.to_csv('states_for_r_chrono.csv')